# Assignment

<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
   <p>
One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.
</p>
    
   <p>
Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

## Task-1

<font face='georgia'>
    <h4><strong>1. Build a TFIDF Vectorizer & compare its results with Sklearn:</strong></h4>

<ul>
    <li> As a part of this task you will be implementing TFIDF vectorizer on a collection of text documents.</li>
    <br>
    <li> You should compare the results of your own implementation of TFIDF vectorizer with that of sklearns implemenation TFIDF vectorizer.</li>
    <br>
    <li> Sklearn does few more tweaks in the implementation of its version of TFIDF vectorizer, so to replicate the exact results you would need to add following things to your custom implementation of tfidf vectorizer:
       <ol>
        <li> Sklearn has its vocabulary generated from idf sroted in alphabetical order</li>
        <li> Sklearn formula of idf is different from the standard textbook formula. Here the constant <strong>"1"</strong> is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions.
            
 $IDF(t) = 1+\log_{e}\frac{1\text{ }+\text{ Total  number of documents in collection}} {1+\text{Number of documents with term t in it}}.$
        </li>
        <li> Sklearn applies L2-normalization on its output matrix.</li>
        <li> The final output of sklearn tfidf vectorizer is a sparse matrix.</li>
    </ol>
   </ul>

### Corpus

In [7]:
## SkLearn# Collection of string documents

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### SkLearn Implementation

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

<class 'sklearn.feature_extraction.text.TfidfVectorizer'>


In [3]:
# sklearn feature names, they are sorted in alphabetic order by default.

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


C:\Users\prana\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [259]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [260]:
# shape of sklearn tfidf vectorizer output after applying transform method.

skl_output.shape

(4, 9)

In [261]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [262]:
# sklearn tfidf values for first line of the above corpus.
# To understand the output better, here we are converting the sparse output matrix to dense matrix and printing it.
# Notice that this output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### custom implementation

In [263]:
# Write your code here.
# Make sure its well documented and readble with appropriate comments.
# Compare your results with the above sklearn tfidf vectorizer
# You are not supposed to use any other library apart from the ones given below

from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np

In [264]:
print(corpus)
print("length of corpus is :",len(corpus))

['this is the first document', 'this document is the second document', 'and this is the third one', 'is this the first document']
length of corpus is : 4


In [265]:
#Generate the idf values of all the unique words, at first , fit() function is called and inside that IDF() function is called
def IDF(corpus,unique_words):
    N=len(corpus)
    dict_idf={}
    for word in unique_words:
        count=0
        for row in corpus:
            if word in row:
                count=count+1
        dict_idf[word]=math.log((1+N)/(count+1))+1
    return dict_idf


# Implementation of the fit() method
def fit(corpus):
    unique_words=[]
    for row in corpus:
        for word in row.split():
            if len(word) >= 2 and word not in unique_words:
                unique_words.append(word)
            else:
                continue
    # Sort the unique words in the dataset
    unique_words.sort()
    
    # Generate a dictonary of word
    vocabulary={j:i for i,j in enumerate(unique_words)}
    idf_values=IDF(corpus,unique_words)
    return vocabulary,idf_values
Vocabulary, idf_of_vocabulary = fit(corpus)

In [266]:
print(list(Vocabulary))

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [267]:
list(Vocabulary) == vectorizer.get_feature_names()    # 2 lists are compared

True

<b>Note:</b> 'list(Vocabulary)' is custom implemented and 'vectorizer.get_feature_names()' is sklearn implemented<br>
<b>Observation:</b>
custom implementation is same as tfidf sklearn implementation

In [268]:
print(idf_of_vocabulary,"\n")
print(idf_of_vocabulary.values())

{'and': 1.916290731874155, 'document': 1.2231435513142097, 'first': 1.5108256237659907, 'is': 1.0, 'one': 1.916290731874155, 'second': 1.916290731874155, 'the': 1.0, 'third': 1.916290731874155, 'this': 1.0} 

dict_values([1.916290731874155, 1.2231435513142097, 1.5108256237659907, 1.0, 1.916290731874155, 1.916290731874155, 1.0, 1.916290731874155, 1.0])


In [278]:
def transform(corpus,Vocabulary,idf_of_vocabulary):
    #sparse_matrix = csr_matrix((len(corpus), len(Vocabulary)), dtype = np.float64).toarray()
    sparse_matrix = csr_matrix((len(corpus), len(Vocabulary)), dtype = np.float64)
    
    for row  in range(0, len(corpus)):
        number_of_words_in_row = Counter(corpus[row].split())   #will generate dictionary of words with no of repetetions 
        for word in corpus[row].split():
            if word in list(Vocabulary.keys()):
                tf_idf_value = (number_of_words_in_row[word] / len(corpus[row].split())) * (idf_of_vocabulary[word])
                sparse_matrix[row, Vocabulary[word]] = tf_idf_value
    print('Before normalising:\n',sparse_matrix,'\n')
    output = normalize(sparse_matrix, norm = 'l2', axis = 1, copy = True, return_norm = False)
    return output

final_output = transform(corpus, Vocabulary, idf_of_vocabulary)
print("Normalized Sparse Matrix:\n",final_output)
print("\n\nNormalized Sparse Matrix in array format:\n", final_output.toarray())
print('\nShape is :',final_output.shape) 

Before normalising:
   (19, 4)	0.03492101561501877
  (20, 2)	0.3777064059414977
  (24, 4)	0.638763577291385
  (38, 4)	0.10646059621523084
  (106, 4)	0.1916290731874155
  (134, 4)	0.01807821445164297
  (135, 2)	0.004329013248613154
  (135, 4)	0.027454021946621134
  (137, 4)	0.1368779094195825
  (148, 4)	0.014628173525756908
  (161, 4)	0.273755818839165
  (191, 4)	0.05323029810761542
  (193, 5)	0.08331698834235457
  (202, 2)	0.18885320297074884
  (231, 4)	0.10646059621523084
  (231, 5)	0.10646059621523084
  (250, 4)	0.15969089432284625
  (256, 4)	0.21292119243046168
  (260, 4)	0.2395363414842694
  (284, 4)	0.14740697937493502
  (306, 5)	0.3193817886456925
  (317, 2)	0.10791611598328504
  (325, 4)	0.100857406940745
  (336, 2)	0.18885320297074884
  (342, 5)	0.1197681707421347
  :	:
  (582, 5)	0.3193817886456925
  (584, 4)	0.100857406940745
  (587, 4)	0.2395363414842694
  (588, 4)	0.383258146374831
  (592, 4)	0.21292119243046168
  (606, 4)	0.638763577291385
  (613, 4)	0.14740697937493502
  

In [279]:
a=final_output.toarray()
print('cusom implemented matrix:\n',a[0])
print("sklearn 1st corpus matrix:\n",skl_output[0].toarray())
#np.array_equal(skl_output[0].toarray(),a[0],equal_nan=True)

cusom implemented matrix:
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
sklearn 1st corpus matrix:
 [[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


<b><i>When comparing my implementation of TfidfVectorizer with sklearns implementation the results will match.</i></b>

## Task-2

<font face='georgia'>
    <h4><strong>2. Implement max features functionality:</strong></h4>

<ul>
    <li> As a part of this task you have to modify your fit and transform functions so that your vocab will contain only 50 terms with top idf scores.</li>
    <br>
    <li>This task is similar to your previous task, just that here your vocabulary is limited to only top 50 features names based on their idf values. Basically your output will have exactly 50 columns and the number of rows will depend on the number of documents you have in your corpus.</li>
    <br>
    <li>Here you will be give a pickle file, with file name <strong>cleaned_strings</strong>. You would have to load the corpus from this file and use it as input to your tfidf vectorizer.</li>
    <br>
   </ul>

In [280]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type

import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print(corpus[:10])
print("\nNumber of documents in corpus = ",len(corpus))

['slow moving aimless movie distressed drifting young man', 'not sure lost flat characters audience nearly half walked', 'attempting artiness black white clever camera angles movie disappointed became even ridiculous acting poor plot lines almost non existent', 'little music anything speak', 'best scene movie gerardo trying find song keeps running head', 'rest movie lacks art charm meaning emptiness works guess empty', 'wasted two hours', 'saw movie today thought good effort good messages kids', 'bit predictable', 'loved casting jimmy buffet science teacher']

Number of documents in corpus =  746


In [281]:
# Write your code here.
# Try not to hardcode any values.
# Make sure its well documented and readble with appropriate comments.

In [282]:
#Generate the idf values of all the unique words
def IDF(corpus,unique_words):
    N=len(corpus)
    dict_idf={}
    for word in unique_words:
        count=0
        for row in corpus:
            if word in row.split(" "):
                count=count+1
                dict_idf[word]=(math.log((1+N)/(count+1)))+1
    return dict_idf


# Implementation of the fit() method
def fit(corpus):
    unique_words=[]
    for row in corpus:
        for word in row.split(" "):
            if len(word) >= 2 and word not in unique_words:
                unique_words.append(word)
            else:
                continue
    # Sort the unique words in the dataset
    #unique_words.sort()
    
    idf_values=IDF(corpus,unique_words)

    #below line , I have converted to descending order from slicing operation since 'np.argsort' will sort in ascending order 
    top50_idf_value_indices=np.argsort(list(idf_values.values()))[-1:-51:-1]    #https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
    print(len(list(top50_idf_value_indices)))
    top50_idf_values = np.take(list(idf_values.values()), top50_idf_value_indices)       #https://numpy.org/doc/stable/reference/generated/numpy.take.html
    top50_idf_words = np.take(list(idf_values.keys()), top50_idf_value_indices)

    idf_values_50=dict(zip(top50_idf_words,top50_idf_values))
    vocabulary_50={j:i for i,j in enumerate(list(idf_values_50.keys()))}
    return vocabulary_50,idf_values_50

vocabulary_50, idf_values_50 = fit(corpus)

50


In [283]:
print(idf_values_50)

{'exceptionally': 6.922918004572872, 'hatred': 6.922918004572872, 'curtain': 6.922918004572872, 'ive': 6.922918004572872, 'edition': 6.922918004572872, 'lid': 6.922918004572872, 'cant': 6.922918004572872, 'romantic': 6.922918004572872, 'junkyard': 6.922918004572872, 'laughed': 6.922918004572872, 'chosen': 6.922918004572872, 'tortured': 6.922918004572872, 'disgusting': 6.922918004572872, 'blatant': 6.922918004572872, 'propaganda': 6.922918004572872, 'admitted': 6.922918004572872, 'spot': 6.922918004572872, 'elsewhere': 6.922918004572872, 'school': 6.922918004572872, 'pile': 6.922918004572872, 'round': 6.922918004572872, 'enter': 6.922918004572872, 'regrettable': 6.922918004572872, 'thunderbirds': 6.922918004572872, 'motion': 6.922918004572872, 'needlessly': 6.922918004572872, 'repeats': 6.922918004572872, 'backed': 6.922918004572872, 'vehicles': 6.922918004572872, 'corn': 6.922918004572872, 'caught': 6.922918004572872, 'funniest': 6.922918004572872, 'hes': 6.922918004572872, 'yelps': 6.

In [284]:
def transform(corpus,Vocabulary,idf_of_vocabulary):
    #sparse_matrix = csr_matrix((len(corpus), len(Vocabulary)), dtype = np.float64).toarray()
    sparse_matrix = csr_matrix((len(corpus), len(Vocabulary)), dtype = np.float64)
    
    for row  in range(0, len(corpus)):
        number_of_words_in_row = Counter(corpus[row].split())
        for word in corpus[row].split():
            if word in list(Vocabulary.keys()):
                tf_idf_value = (number_of_words_in_row[word] / len(corpus[row].split())) * (idf_of_vocabulary[word])
                sparse_matrix[row, Vocabulary[word]] = tf_idf_value
    print('Before normalising:\n',sparse_matrix,'\n')
    output = normalize(sparse_matrix, norm = 'l2', axis = 1, copy = True, return_norm = False)
    return output

final_output = transform(corpus, vocabulary_50, idf_values_50)
print("Normalized Sparse Matrix:\n",final_output)
print('\nShape is :',final_output.shape) 

Before normalising:
   (205, 34)	1.3845836009145744
  (205, 35)	1.3845836009145744
  (205, 36)	1.3845836009145744
  (205, 37)	1.3845836009145744
  (209, 38)	1.3845836009145744
  (210, 39)	1.3845836009145744
  (210, 40)	1.3845836009145744
  (211, 41)	1.3845836009145744
  (213, 42)	1.3845836009145744
  (213, 43)	1.3845836009145744
  (215, 44)	1.3845836009145744
  (216, 45)	0.4615278669715248
  (216, 46)	0.4615278669715248
  (216, 47)	0.4615278669715248
  (216, 48)	0.4615278669715248
  (216, 49)	0.4615278669715248
  (222, 2)	0.20361523542861387
  (222, 3)	0.20361523542861387
  (222, 4)	0.20361523542861387
  (222, 5)	0.20361523542861387
  (222, 30)	0.20361523542861387
  (222, 31)	0.20361523542861387
  (222, 32)	0.20361523542861387
  (222, 33)	0.20361523542861387
  (223, 6)	0.3146780911169487
  (223, 7)	0.3146780911169487
  (223, 8)	0.6293561822338974
  (223, 9)	0.3146780911169487
  (224, 10)	0.9889882863675531
  (224, 11)	0.9889882863675531
  (224, 12)	0.9889882863675531
  (224, 13)	0.9889

In [285]:
print(final_output[205])

  (0, 34)	0.5
  (0, 35)	0.5
  (0, 36)	0.5
  (0, 37)	0.5


In [277]:
print(final_output[205].toarray())

[[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5
  0.5 0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]]
